In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"malayajrath","key":"4489fc1f7348b0fe84eea8dea733bd1d"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

new-plant-diseases-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
from zipfile import ZipFile
name='new-plant-diseases-dataset.zip'
with ZipFile(name,'r') as zip:
  zip.extractall()

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
training_set = train_datagen.flow_from_directory('/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 70295 images belonging to 38 classes.


In [9]:
valid_set = valid_datagen.flow_from_directory("/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
                                              target_size=(224,224),
                                              batch_size=64,
                                              class_mode='categorical'
                                              )

Found 17572 images belonging to 38 classes.


In [10]:
class_dict = training_set.class_indices

In [11]:
class_dict

{'Apple___Apple_scab': 0,
 'Apple___Black_rot': 1,
 'Apple___Cedar_apple_rust': 2,
 'Apple___healthy': 3,
 'Blueberry___healthy': 4,
 'Cherry_(including_sour)___Powdery_mildew': 5,
 'Cherry_(including_sour)___healthy': 6,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7,
 'Corn_(maize)___Common_rust_': 8,
 'Corn_(maize)___Northern_Leaf_Blight': 9,
 'Corn_(maize)___healthy': 10,
 'Grape___Black_rot': 11,
 'Grape___Esca_(Black_Measles)': 12,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13,
 'Grape___healthy': 14,
 'Orange___Haunglongbing_(Citrus_greening)': 15,
 'Peach___Bacterial_spot': 16,
 'Peach___healthy': 17,
 'Pepper,_bell___Bacterial_spot': 18,
 'Pepper,_bell___healthy': 19,
 'Potato___Early_blight': 20,
 'Potato___Late_blight': 21,
 'Potato___healthy': 22,
 'Raspberry___healthy': 23,
 'Soybean___healthy': 24,
 'Squash___Powdery_mildew': 25,
 'Strawberry___Leaf_scorch': 26,
 'Strawberry___healthy': 27,
 'Tomato___Bacterial_spot': 28,
 'Tomato___Early_blight': 29,
 'Toma

In [12]:
model= tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=tf.keras.Input(shape=(224,224,3))
)

In [13]:

model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [14]:
for layer in model.layers:
  layer.trainable = False

In [15]:
base_input = model.layers[0].input
base_output = model.get_layer('mixed7').output

In [16]:
x = tf.keras.layers.Flatten()(base_output)
outputs=tf.keras.layers.Dense(38,activation="softmax")(x)

In [17]:
my_model = tf.keras.Model(inputs=base_input,outputs=outputs)

In [18]:
my_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [19]:
my_model.compile(tf.keras.optimizers.Adam(),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

In [20]:
my_model.fit(training_set,validation_data=valid_set,epochs=6)

Epoch 1/6
1099/1099 [==============================] - 870s 786ms/step - loss: 1.0751 - accuracy: 0.7561 - val_loss: 0.5313 - val_accuracy: 0.9041
Epoch 2/6
1099/1099 [==============================] - 874s 795ms/step - loss: 0.5372 - accuracy: 0.9046 - val_loss: 0.6705 - val_accuracy: 0.9083
Epoch 3/6
1099/1099 [==============================] - 866s 788ms/step - loss: 0.5331 - accuracy: 0.9203 - val_loss: 0.4486 - val_accuracy: 0.9384
Epoch 4/6
1099/1099 [==============================] - 860s 783ms/step - loss: 0.4863 - accuracy: 0.9326 - val_loss: 0.4914 - val_accuracy: 0.9416
Epoch 5/6
1099/1099 [==============================] - 869s 791ms/step - loss: 0.4748 - accuracy: 0.9404 - val_loss: 0.4979 - val_accuracy: 0.9500
Epoch 6/6
1099/1099 [==============================] - 857s 780ms/step - loss: 0.4775 - accuracy: 0.9431 - val_loss: 0.5406 - val_accuracy: 0.9466


In [21]:
my_model.save("model")

INFO:tensorflow:Assets written to: model/assets


In [22]:
my_model.save("model.h5")

In [23]:
ld_model = tf.keras.models.load_model("/content/model.h5")

In [24]:
import pickle as pkl

In [25]:
with open("label.pkl","wb") as file:
  pkl.dump(class_dict,file)

In [26]:
with open("/content/label.pkl","rb") as file:
  lis=pkl.load(file)
  print(lis)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato___Bacterial_spot': 28, 'Tomato___Early_blight': 29, 'Tomato___Late_blight': 30, 'Tomato

In [30]:
from tensorflow.keras.preprocessing import image
import numpy as np


In [44]:
img = image.load_img("/content/test/test/AppleCedarRust1.JPG",target_size=(224,224))

In [45]:
img = image.img_to_array(img)
img.shape
img = np.expand_dims(img,axis=0)
img.shape

(1, 224, 224, 3)

In [46]:
img = img/255

In [47]:
pred = ld_model.predict(img)

In [48]:
pred

array([[2.2625159e-25, 1.0312264e-33, 1.0000000e+00, 0.0000000e+00,
        2.3732959e-32, 0.0000000e+00, 1.2408649e-38, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        3.4947137e-37, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 1.5483514e-38, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [49]:
pred=pred.flatten()

In [53]:
pred

array([2.2625159e-25, 1.0312264e-33, 1.0000000e+00, 0.0000000e+00,
       2.3732959e-32, 0.0000000e+00, 1.2408649e-38, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       3.4947137e-37, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 1.5483514e-38, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [51]:
idx=np.argmax(pred)

In [52]:
idx

2

In [57]:
ans = ""
with open("/content/label.pkl","rb") as file:
  lis=pkl.load(file)
  for class_name in lis:
    if lis[class_name]==idx:
      ans=class_name

print(ans)
    

Apple___Cedar_apple_rust
